In [1]:
import glob
import os

import btrack
import numpy as np

from skimage.io import imread

from tqdm import tqdm

In [2]:
import sys
sys.path.append('../../cellx')
from cellx.tools.image import InfinitePaddedImage

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

import logging

logger = tf.get_logger()
logger.setLevel(logging.WARNING)

In [24]:
PATH = '/home/nathan/data/kraken/commitment/MK0000'#"/media/quantumjot/Data/Manasi/MK0000/"
POSITION = 'Pos0'#"Position1"
CHANNELS = ("channel001", )
channel = CHANNELS[0]
INPUT_SHAPE = (64, 64, 3)

In [20]:
i = 22222
print(f'{i:09}')

000022222


In [26]:
def segmentation_generator(position, channel):
    files = [
        os.path.join(PATH, position, f'{position}_cp_masks', f'img_{channel}_position00{position[-1]}_time{i:09}_z000_cp_masks.png') 
        for i in range(15)
    ]
    
    for file in files:
        yield imread(file)

# def segmentation_generator(position):
#     files = [
#         os.path.join(PATH, position, "cellpose", f"cellpose_{position}_{i}_cp_masks.png") 
#         for i in range(150)
#     ]
    
#     for file in files:
#         yield imread(file)

In [27]:
objects = btrack.utils.segmentation_to_objects(
    segmentation_generator(POSITION, channel), 
    properties = ('area', ), 
)

[INFO][2021/10/11 04:20:08 PM] Localizing objects from segmentation...
[INFO][2021/10/11 04:20:13 PM] Objects are of type: <class 'dict'>
[INFO][2021/10/11 04:20:13 PM] ...Found 3795 objects in 15 frames.


In [28]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,779.193443,5.17377,0.0,0,False,0,5,0.0,305


In [8]:
# objects = objects[:100]

In [29]:
def feature_generator(position, channel, objects):
    stack = imread(os.path.join(PATH, position, f'{position}_stacks', f'{channel}.tif'))
    
    win = INPUT_SHAPE[0] // 2
    
    for obj in tqdm(objects):
        idx = obj.t
        frame = InfinitePaddedImage(stack[idx, ...])
        
        sx = slice(int(obj.x)-win, int(obj.x)+win, 1)
        sy = slice(int(obj.y)-win, int(obj.y)+win, 1)
        
        crop = frame[sy, sx]
        
        yield np.stack([crop]*3, axis=-1)[np.newaxis, ...]

# def feature_generator(position, channel, objects):
#     stack = imread(os.path.join(PATH, position, f"{channel}_Concatenated.tif"))
    
#     win = INPUT_SHAPE[0] // 2
    
#     for obj in tqdm(objects):
#         idx = obj.t
#         frame = InfinitePaddedImage(stack[idx, ...])
        
#         sx = slice(int(obj.x)-win, int(obj.x)+win, 1)
#         sy = slice(int(obj.y)-win, int(obj.y)+win, 1)
        
#         crop = frame[sy, sx]
        
#         yield np.stack([crop]*3, axis=-1)[np.newaxis, ...]

In [30]:
model = ResNet50(
    include_top=False,         
    weights="imagenet",        
    input_shape=INPUT_SHAPE,
    pooling='avg',
)

94781440/94765736 [==============================] - 1s 0us/step


In [31]:
y = model.predict(feature_generator(POSITION, CHANNELS[0], objects))

100%|██████████| 3795/3795 [01:28<00:00, 43.00it/s]


In [12]:
y.shape

(15446, 2048)

In [32]:
assert y.shape[0] == len(objects)

In [33]:
for i, obj in enumerate(objects):
    obj.properties = {'resnet-features': y[i, ...]}

In [38]:
objects

[{'ID': 0, 'x': 779.1934426229508, 'y': 5.173770491803278, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 5, 'prob': 0.0, 'area': 305, 'resnet-features': array([0.       , 0.       , 0.       , ..., 2.4860282, 0.       ,
        0.       ], dtype=float32)},
 {'ID': 1, 'x': 21.510807736063708, 'y': 10.617747440273037, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 5, 'prob': 0.0, 'area': 879, 'resnet-features': array([0.       , 0.       , 1.616944 , ..., 1.2816163, 1.222204 ,
        0.       ], dtype=float32)},
 {'ID': 2, 'x': 340.0581915846016, 'y': 12.949865711727842, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 5, 'prob': 0.0, 'area': 1117, 'resnet-features': array([0.        , 0.        , 0.        , ..., 1.7987564 , 0.05061233,
        0.        ], dtype=float32)},
 {'ID': 3, 'x': 1461.55, 'y': 12.47, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 5, 'prob': 0.0, 'area': 1100, 'resnet-features': array([0.       , 0.       , 0.       , ..., 2.

In [48]:
objects[0]

ValueError: Shape of passed values is (2048, 11), indices imply (1, 11)

{'ID': 0, 'x': 779.1934426229508, 'y': 5.173770491803278, 'z': 0.0, 't': 0, 'dummy': False, 'states': 0, 'label': 5, 'prob': 0.0, 'area': 305, 'resnet-features': array([0.       , 0.       , 0.       , ..., 2.4860282, 0.       ,
       0.       ], dtype=float32)}

In [49]:
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/cell_config.json')
    tracker.max_search_radius = 30

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume
    tracker.volume=((0, 512), (0, 512), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking_with_resnet_feat.h5'), obj_type='obj_type_1')
    
    tracks = tracker.tracks

[INFO][2021/10/11 04:36:57 PM] Loaded btrack: /home/nathan/src/btrack/btrack/libs/libtracker.so
[INFO][2021/10/11 04:36:57 PM] btrack (v0.4.2) library imported
[INFO][2021/10/11 04:36:57 PM] Setting max XYZ search radius to: 100
[INFO][2021/10/11 04:36:57 PM] Starting BayesianTracker session
[INFO][2021/10/11 04:36:57 PM] Loading configuration file: /home/nathan/analysis/cell-comp-analysis/BayesianTracker/models/cell_config.json
[INFO][2021/10/11 04:36:57 PM] Loading motion model: b'cell_motion'
[INFO][2021/10/11 04:36:57 PM] Setting max XYZ search radius to: 30
[INFO][2021/10/11 04:36:57 PM] Objects are of type: <class 'list'>
[INFO][2021/10/11 04:36:57 PM] Set volume to ((0, 512), (0, 512), (-100000.0, 100000.0))
[INFO][2021/10/11 04:36:57 PM] Starting tracking... 
[INFO][2021/10/11 04:36:57 PM] Tracking objects in frames 0 to 15 (of 15)...
[INFO][2021/10/11 04:36:57 PM]  - Timing (Bayesian updates: 3.11ms, Linking: 0.32ms)
[INFO][2021/10/11 04:36:57 PM]  - Probabilities (Link: 0.657

In [55]:
tracks[1]

ValueError: Data must be 1-dimensional

OrderedDict([('ID', 159), ('t', [0]), ('x', [1501.3949343339586]), ('y', [97.18386491557223]), ('z', [0.0]), ('parent', 159), ('root', 159), ('state', [5]), ('generation', 0), ('dummy', [False]), ('area', array([1066])), ('resnet-features', array([[0.        , 0.        , 0.        , ..., 0.17739877, 0.        ,
        0.        ]], dtype=float32))])

In [59]:
tracks[0]

ValueError: Data must be 1-dimensional

OrderedDict([('ID', 174), ('t', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]), ('x', [63.128093158660846, 69.56949394155382, 73.11236749116608, 70.57414170927684, 72.33926128590971, 68.93254520166899, 68.28112712975098, 70.00959561343386, 72.3612676056338, 61.39054545454545, 70.80423280423281, 72.82648401826484, 69.35944055944056, 78.36260053619303, 69.98549723756906]), ('y', [84.93959243085881, 90.6735566642908, 94.539222614841, 98.44558071585098, 101.28796169630643, 104.19888734353269, 106.90366972477064, 109.43385880740233, 111.52535211267606, 112.11854545454545, 114.14087301587301, 113.84931506849315, 110.8881118881119, 111.02144772117963, 107.90331491712708]), ('z', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), ('parent', 174), ('root', 174), ('state', [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), ('generation', 0), ('dummy', [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]), ('area', 